In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.preprocessing import StandardScaler 
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib
import time
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

In [2]:
sup_min = 10
sup_max = 300
lat_min = -35
lat_max = -34
lon_min = -59
lon_max = -58
precio_min = 30000
precio_max = 3000000

propiedades = pd.read_csv('../../datos_con_fecha.csv')

propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2017)\
                        & (propiedades.price_aprox_usd <= precio_max) & (propiedades.price_aprox_usd >= precio_min) &\
                         (propiedades.superficie <= sup_max) & (propiedades.superficie >= sup_min) &\
                      (propiedades.lat <= lat_max) & (propiedades.lat >= lat_min) &\
                       (propiedades.lon <= lon_max) & (propiedades.lon >= lon_min),:]

columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', \
            'pileta', 'cochera','transporte','comercio', 'servicios', 'doble piso', 'jardin']

set_entrenamiento_datos = propiedades.loc[:,columnas]
set_entrenamiento_resultado = propiedades.loc[:,'price_aprox_usd']

In [3]:
tipos = preprocessing.OneHotEncoder()
tipos.fit([[0],[1],[2], [3]])

OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=True)

In [4]:
set_entrenamiento_datos.loc[:,'property_type_encoded'] = set_entrenamiento_datos.loc[:,'property_type_encoded']\
.apply(lambda x: list(tipos.transform([[x]]).toarray()[0]))

In [5]:
set_pruebas = pd.read_csv("../properati_dataset_modificado2.csv")

set_pruebas.loc[:,'property_type_encoded'] = set_pruebas.loc[:,'property_type_encoded']\
.apply(lambda x: list(tipos.transform([[x]]).toarray()[0]))

In [12]:
def predecir(x, estimador):
    #list(x[:3]) + x[3]
    x = list(x[:3]) + x[3] + list(x[4:])
    return estimador.predict([x])

In [7]:
set_entrenamiento_datos.reset_index(drop = True, inplace = True)

In [9]:
datos = []

for i in range(len(set_entrenamiento_datos)): 
    x = []
    for n in columnas:
        x.append(set_entrenamiento_datos[n][i])
    x = x[:3] + x[3] + x[4:]
    datos.append(x)

# Decision tree con gridsearch

In [21]:
criterios = ['mse','friedman_mse']
max_depth = [None,50,75,100,150]
features = [0.4,0.6,0.8,1.0]

parametros = {"criterion" : criterios, "max_depth" : max_depth, "max_features" : features}

iteraciones_cross_validation = 20
arbol = DecisionTreeRegressor()

In [22]:
inicio = time.strftime("%X")

grid = GridSearchCV( estimator = arbol, param_grid = parametros, n_jobs = 2, cv = iteraciones_cross_validation)

grid.fit(datos, set_entrenamiento_resultado)

score = grid.best_score_ * 100
mejores_parametros = grid.best_params_
fin = time.strftime("%X")

print("Tiempo: {} --- {} \n Precision: {:.2f} \n Parametros = {}".format(inicio,fin,score,mejores_parametros))

Tiempo: 14:21:30 --- 17:09:45 
 Precision: 96.32 
 Parametros = {'max_depth': 75, 'max_features': 1.0, 'criterion': 'mse'}


In [23]:
set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,columnas].apply(lambda x: predecir(x, grid)[0],axis = 1)

resultado = set_pruebas.loc[:,['id','price_usd']]

resultado.to_csv('resultados/Decision_tree_gridsearch_con_indice.csv', index = False)

# Extra trees con gridsearch

In [24]:
criterios = ['mse']
max_depth = [None,100]
features = [0.4,0.6,1.0]
estimators = [20, 30,50]

parametros = {"criterion" : criterios, "max_depth" : max_depth, \
              "n_estimators" : estimators, "max_features" : features}

iteraciones_cross_validation = 10
arbol = ExtraTreesRegressor()

In [25]:
inicio = time.strftime("%X")

grid = GridSearchCV( estimator = arbol, param_grid = parametros, n_jobs = 2, cv = iteraciones_cross_validation)

grid.fit(datos, set_entrenamiento_resultado)

score = grid.best_score_ * 100
mejores_parametros = grid.best_params_
fin = time.strftime("%X")

print("Tiempo: {} --- {} \n Precision: {:.2f} \n Parametros = {}".format(inicio,fin,score,mejores_parametros))

Tiempo: 17:21:24 --- 18:29:07 
 Precision: 97.57 
 Parametros = {'max_depth': 100, 'max_features': 0.4, 'n_estimators': 50, 'criterion': 'mse'}


In [26]:
set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,columnas].apply(lambda x: predecir(x, grid)[0],axis = 1)

resultado = set_pruebas.loc[:,['id','price_usd']]

resultado.to_csv('resultados/Extra_trees_gridsearch_con_indice.csv', index = False)

# Utilizando menos columnas

In [77]:
propiedades = pd.read_csv('../../set_datos_one_hot_encoder.csv')
set_pruebas = pd.read_csv('../set_pruebas_one_hot_encoder.csv')

columnas = ['superficie','lat','lon','property_type_encoded','seguridad','gimnasio','aire','pileta','cochera']

In [48]:
def encodear(x):
    return [float(x) for x in x.strip('[').strip(']').split(',')]

In [78]:
propiedades.loc[:,'property_type_encoded'] = propiedades.loc[:,'property_type_encoded']\
.apply(lambda x: encodear(x))

propiedades.loc[:,'state_name_encoded'] = propiedades.loc[:,'state_name_encoded']\
.apply(lambda x: encodear(x))

set_pruebas.loc[:,'property_type_encoded'] = set_pruebas.loc[:,'property_type_encoded']\
.apply(lambda x: encodear(x))

set_pruebas.loc[:,'state_name_encoded'] = set_pruebas.loc[:,'state_name_encoded']\
.apply(lambda x: encodear(x))

In [83]:
def predecir(x, estimador):
    x = list(x[:3]) + x[3] + list(x[4:])
    return estimador.predict([x])

In [84]:
set_entrenamiento_datos = propiedades.loc[:,columnas]
set_entrenamiento_resultado = propiedades.loc[:,'price_aprox_usd']
#set_entrenamiento_datos.reset_index(drop = True, inplace = True)

In [85]:
datos = []

for i in range(len(set_entrenamiento_datos)): 
    x = []
    for n in columnas:
        x.append(set_entrenamiento_datos[n][i])
    x = x[:3] + x[3] + x[4:]
    datos.append(x)

In [87]:
criterios = ['mse']
max_depth = [None,100,150]
features = [0.6,0.8,1.0]

parametros = {"criterion" : criterios, "max_depth" : max_depth, "max_features" : features}

iteraciones_cross_validation = 5
arbol = DecisionTreeRegressor()

In [88]:
inicio = time.strftime("%X")

grid = GridSearchCV( estimator = arbol, param_grid = parametros, n_jobs = 2, cv = iteraciones_cross_validation)

grid.fit(datos, set_entrenamiento_resultado)

score = grid.best_score_ * 100
mejores_parametros = grid.best_params_
fin = time.strftime("%X")

print("Tiempo: {} --- {} \n Precision: {:.2f} \n Parametros = {}".format(inicio,fin,score,mejores_parametros))

Tiempo: 03:00:32 --- 03:05:03 
 Precision: 96.19 
 Parametros = {'criterion': 'mse', 'max_depth': 150, 'max_features': 1.0}


In [89]:
set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,columnas].apply(lambda x: predecir(x, grid)[0],axis = 1)

resultado = set_pruebas.loc[:,['id','price_usd']]

resultado.to_csv('resultados/Decision_tree_HotEncoder_menos_columnas.csv', index = False)

# Usando el state name encoded

In [90]:
propiedades = pd.read_csv('../../set_datos_one_hot_encoder.csv')
set_pruebas = pd.read_csv('../set_pruebas_one_hot_encoder.csv')

columnas = ['superficie','lat','lon','property_type_encoded', 'state_name_encoded',\
            'seguridad','gimnasio','aire','pileta','cochera']

In [91]:
def encodear(x):
    return [float(x) for x in x.strip('[').strip(']').split(',')]

In [92]:
propiedades.loc[:,'property_type_encoded'] = propiedades.loc[:,'property_type_encoded']\
.apply(lambda x: encodear(x))

propiedades.loc[:,'state_name_encoded'] = propiedades.loc[:,'state_name_encoded']\
.apply(lambda x: encodear(x))

set_pruebas.loc[:,'property_type_encoded'] = set_pruebas.loc[:,'property_type_encoded']\
.apply(lambda x: encodear(x))

set_pruebas.loc[:,'state_name_encoded'] = set_pruebas.loc[:,'state_name_encoded']\
.apply(lambda x: encodear(x))

In [93]:
def predecir(x, estimador):
    x = list(x[:3]) + x[3] + x[4] + list(x[5:])
    return estimador.predict([x])

In [94]:
set_entrenamiento_datos = propiedades.loc[:,columnas]
set_entrenamiento_resultado = propiedades.loc[:,'price_aprox_usd']
#set_entrenamiento_datos.reset_index(drop = True, inplace = True)

In [95]:
datos = []

for i in range(len(set_entrenamiento_datos)): 
    x = []
    for n in columnas:
        x.append(set_entrenamiento_datos[n][i])
    x = x[:3] + x[3] + x[4] + x[5:]
    datos.append(x)

In [96]:
criterios = ['mse']
max_depth = [None,100,150]
features = [0.6,0.8,1.0]

parametros = {"criterion" : criterios, "max_depth" : max_depth, "max_features" : features}

iteraciones_cross_validation = 5
arbol = DecisionTreeRegressor()

In [97]:
inicio = time.strftime("%X")

grid = GridSearchCV( estimator = arbol, param_grid = parametros, n_jobs = 2, cv = iteraciones_cross_validation)

grid.fit(datos, set_entrenamiento_resultado)

score = grid.best_score_ * 100
mejores_parametros = grid.best_params_
fin = time.strftime("%X")

print("Tiempo: {} --- {} \n Precision: {:.2f} \n Parametros = {}".format(inicio,fin,score,mejores_parametros))

Tiempo: 03:06:31 --- 03:10:57 
 Precision: 96.16 
 Parametros = {'criterion': 'mse', 'max_depth': 100, 'max_features': 0.6}


In [98]:
set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,columnas].apply(lambda x: predecir(x, grid)[0],axis = 1)

resultado = set_pruebas.loc[:,['id','price_usd']]

resultado.to_csv('resultados/Decision_tree_HotEncoder_StateName.csv', index = False)